In [13]:
from Mount.RDP_Outlier_Analysis.src.Transitive_Annotation_Utils import *

def Summarize_Label_Flips(df_wo_adv, df_adv, label_Adversarial_Seq, label):
    df_adv = df_adv[['genus']].rename(columns = {'genus':'G-ADV'})
    df_adv = df_adv.join(df_wo_adv)
    df_filter = df_adv.loc[(df_adv['G-ADV']!=label) & (df_adv['G-WO_ADV']==label)]
    df_filter['Label of Artificial Sequence'] = label_Adversarial_Seq
    return df_filter.reset_index()

def Summarize_Label_Flips_Driver(data_dir, df_wo_adv, label = 'Dorea', cutoff = 100):
    lines = open(data_dir+'summary').readlines()
    summary = []
    for d in lines:
        try: summary.append(eval(d.replace(' nan','-1')))
        except NameError: pass
    df_summary = pd.DataFrame(summary)
    artificial_sequences = df_summary['SeqID'].tolist()
    out = pd.DataFrame()
    
    for i in range(0, cutoff):
        f = data_dir+'Adverserial_Trained_'+artificial_sequences[i]+'.tax.out'
        print(i, f)
        label_artificial_seq = artificial_sequences[i].split("-")[-2]
        df_adv = Parse_RDP_Output(f)
        df_sel = Summarize_Label_Flips(df_wo_adv, df_adv, label_artificial_seq, label)
        df_sel['Num_Artificial_Sequences'] = i+1
        out = out.append(df_sel, ignore_index = True)
        print("=================================================================")
    return out

In [3]:
df_wo_adv = Parse_RDP_Output('Mount-2/projects/RDP_Outlier_Analysis/Gates_Dataset_Experiments/All3000.final.centers.out')
df_wo_adv = df_wo_adv[['genus']].rename(columns = {'genus':'G-WO_ADV'})

In [14]:
data_dir = 'Mount-2/projects/RDP_Outlier_Analysis/RDP_Sensitivity_Analysis/Obfuscating_Taxa/Dorea/'
df_summary = Summarize_Label_Flips_Driver(data_dir, df_wo_adv, cutoff = 25 )

0 Mount-2/projects/RDP_Outlier_Analysis/RDP_Sensitivity_Analysis/Obfuscating_Taxa/Dorea/Adverserial_Trained_2574_1536_3132_5806-seq216_p=8_d=11_pos=497-Mediterraneibacter-Dorea.tax.out


/Users/harihara/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


1 Mount-2/projects/RDP_Outlier_Analysis/RDP_Sensitivity_Analysis/Obfuscating_Taxa/Dorea/Adverserial_Trained_3368_3041_3531_803-seq12_p=1_d=10_pos=448-Mediterraneibacter-Dorea.tax.out
2 Mount-2/projects/RDP_Outlier_Analysis/RDP_Sensitivity_Analysis/Obfuscating_Taxa/Dorea/Adverserial_Trained_2574_1536_5236_1687-seq96_p=4_d=10_pos=99-Mediterraneibacter-Dorea.tax.out
3 Mount-2/projects/RDP_Outlier_Analysis/RDP_Sensitivity_Analysis/Obfuscating_Taxa/Dorea/Adverserial_Trained_2574_1536_3531_803-seq269_p=9_d=11_pos=425-Mediterraneibacter-Dorea.tax.out
4 Mount-2/projects/RDP_Outlier_Analysis/RDP_Sensitivity_Analysis/Obfuscating_Taxa/Dorea/Adverserial_Trained_2574_1536_4814_2434-seq309_p=10_d=10_pos=446-Mediterraneibacter-Dorea.tax.out
5 Mount-2/projects/RDP_Outlier_Analysis/RDP_Sensitivity_Analysis/Obfuscating_Taxa/Dorea/Adverserial_Trained_3368_3041_7327_457-seq229_p=8_d=10_pos=322-Mediterraneibacter-Dorea.tax.out
6 Mount-2/projects/RDP_Outlier_Analysis/RDP_Sensitivity_Analysis/Obfuscating_Tax

In [37]:
df_delta = pd.read_csv('Mount-2/projects/RDP_Outlier_Analysis/Distance_to_Decision_Boundary.txt', sep = "\t")
df_delta = df_delta[df_delta['GB']=='Dorea']
df_dorea = df_delta.groupby(['SB','GA','GB']).agg({'F':'count', 'DB':'mean'})
df_dorea = df_dorea.reset_index()
df_dorea = df_dorea.rename(columns = {'F':'Count', 'DB':'Avg_Delta',
                                      'GA':'Label of Artificial Sequence','SB':'Seq-ID'})
df_dorea = df_dorea.set_index(['Seq-ID' ,'Label of Artificial Sequence'])
print(df_dorea.head())

                                           GB  Count  Avg_Delta
Seq-ID    Label of Artificial Sequence                         
3132_5806 Agathobacter                  Dorea      6       39.0
          Anaerobutyricum               Dorea      1       52.0
          Anaerostipes                  Dorea      2       38.5
          Anaerotignum                  Dorea      1       69.0
          Blautia                       Dorea      5       52.8


In [47]:
df_sel = df_summary.loc[df_summary.groupby(['Seq-ID']).idxmin().Num_Artificial_Sequences].copy()
df_sel = df_sel.set_index(['Seq-ID' ,'Label of Artificial Sequence'])
df_sel = df_sel.join(df_dorea[['Avg_Delta']])
df_sel = df_sel.rename(columns = {'G-ADV':'New Label', 'G-WO_ADV':'Original Label', 
                                  'Num_Artificial_Sequences':'Minimum Number of Transitive Annotations Needed Change Labels',
                                  'Avg_Delta':'Average Distance to the Decision Boundary'})
df_sel = df_sel.reset_index().set_index(['Seq-ID'])
df_sel = df_sel.sort_values(by = ['Average Distance to the Decision Boundary'])

/Users/harihara/anaconda3/lib/python3.7/site-packages/pandas/core/groupby/groupby.py:1309: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  keys, values, mutated = self.grouper.apply(f, data, self.axis)


In [49]:
df_sel = df_sel[['Original Label','New Label', 'Label of Artificial Sequence',
                 'Minimum Number of Transitive Annotations Needed Change Labels',
                 'Average Distance to the Decision Boundary']]
df_sel.to_excel('Dorea-Supplementary-Table.xlsx')